In [1]:
import neo4j
import os

In [2]:
NEO4J_URI = "bolt://0.0.0.0:7687"
NEO4J_USERNAME = input("Your Neo4j username.")
NEO4J_PASSWORD = input("Your Neo4j password.")

In [16]:
ROOT_DIR = "/Users/m.mohammed/Downloads"

In [3]:
os.environ["OPENAI_API_KEY"] = input("Your OPENAI_API_KEY")

In [26]:
os.environ["MISTRAL_API_KEY"] = input("your MISTRAL_API_KEY")

In [6]:
neo4j_driver = neo4j.GraphDatabase.driver(NEO4J_URI,
                                         auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [21]:
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.llm import MistralAILLM


from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings

class Models:
    OPEN_AI = "OPEN_AI"
    MISTRAL_AI = "MISTRAL_AI"

def get_llm(type: str = Models.OPEN_AI):
    llm = None
    if type == Models.MISTRAL_AI:
        llm = MistralAILLM(
            # mistral-large-latest
            model_name="mistral-large-latest",
        )
        llm.invoke("say something")
    else:
        llm = OpenAILLM(
            model_name="gpt-4o-mini",
            model_params={
                "response_format": {"type": "json_object"}, # use json_object formatting for best results
                "temperature": 0 # turning temperature down for more deterministic results
            }
    )
    return llm   

In [8]:
#create text embedder
embedder = OpenAIEmbeddings()

In [9]:
# Basic node labels used in your current domain (politics, etc.)
basic_node_labels = [
    "Person",
    "Position",
    "PoliticalParty",
    "Direction",
    "Value",
    "Goal",
    "Consequence"
]

node_labels = basic_node_labels

# define relationship types
rel_types = ["belongs_to", "had_role", "wanted_role", "has_direction", "has_value",
   "has_influence_to", "introduces_consequences"]

In [10]:
prompt_template = '''
You are a policical, financial, societal, geopolictical researcher tasks with extracting information from papers 
and structuring it in a property graph to inform further political, societal, geopolicitcal, financial and research Q&A.

Extract the entities (nodes) and specify their type from the following Input text.
Also extract the relationships between these nodes. the relationship direction goes from the start node to the end node. 


Return result as JSON using the following format:
{{"nodes": [ {{"id": "0", "label": "the type of entity", "properties": {{"name": "name of entity" }} }}],
  "relationships": [{{"type": "TYPE_OF_RELATIONSHIP", "start_node_id": "0", "end_node_id": "1", "properties": {{"details": "Description of the relationship"}} }}] }}

- Use only the information from the Input text. Do not add any additional information.  
- If the input text is empty, return empty Json. 
- Make sure to create as many nodes and relationships as needed to offer rich medical context for further research.
- An AI knowledge assistant must be able to read this graph and immediately understand the context to inform detailed research questions. 
- Multiple documents will be ingested from different sources and we are using this property graph to connect information, so make sure entity types are fairly general. 

Use only fhe following nodes and relationships (if provided):
{schema}

Assign a unique ID (string) to each node, and reuse it to define relationships.
Do respect the source and target node types for relationship and
the relationship direction.

Do not return any additional information other than the JSON in it.

Examples:
{examples}

Input text:

{text}
'''


In [ ]:
# (Optional) Specify your own Neo4j schema
neo4j_schema = """
Node properties:
Person {name: STRING, chunk_index: INTEGER}
Direction {name: STRING, chunk_index: INTEGER}
PoliticalParty {name: STRING, chunk_index: INTEGER}
Political Party {name: STRING, chunk_index: INTEGER}
Position {name: STRING, chunk_index: INTEGER}
Consequence {name: STRING, chunk_index: INTEGER}
Value {name: STRING, chunk_index: INTEGER}
Chunk {index: INTEGER, text: STRING}
Country {name: French, chunk_index: INTEGER}
Document {path: STRING, createdAt: STRING}
EconomicCondition {name: String, chunk_index: INTEGER}
Region {name, chunk_index, INTEGER}
Resource {name: STRING, chunk_index: INTEGER}
Threat {name, STRNG, chunk_index: INTEGER}
Topic {name: STRING, chunk_index: INTEGER}


Relationship properties:
FROM_CHUNK
FROM_DOCUMENT 
NEXT_CHUNK
belongs_to
had_role {details: STRING}
has_direction
has_goal {details: STRING}
has_influence_to {details: STRING}
has_role
has_value {details: STRING}
introduces_consequences {details: STRING}
wanted_role {details: STRING}

The relationships:
(:Person)-[:has_role]->(:President)
(:Person)-[:belongs_to]->(:Political Party)
(:Political Party)-[:has_direction]->(:Direction)
(:Direction)-[:has_value]->(:Value)
(:Person)-[:has_influence_to]->(:Goal)
(:Position)-[:has_influence_to]->(PoliticalParty)
(:__Entity__)-[:has_value]->(:Value)
(:Goal)-[:introduces_consequences]->(:Topic)
(:Goal)-[:introduces_consequences]->(:Europe)
(:Goal)-[:introduces_consequences]->(:Position)
(:Person)-[:from_chunk]->(:Chunk)
(:Chunk)-[:from_document]->(:Document)
"""

In [32]:
from neo4j_graphrag.experimental.components.text_splitters.fixed_size_splitter import FixedSizeSplitter
from neo4j_graphrag.experimental.pipeline.kg_builder import SimpleKGPipeline

kg_builder_pdf = SimpleKGPipeline(
   llm=get_llm(type=Models.MISTRAL_AI),
   driver=neo4j_driver,
   text_splitter=FixedSizeSplitter(chunk_size=500, chunk_overlap=100),
   embedder=embedder,
   entities=node_labels,
   relations=rel_types,
   prompt_template=prompt_template,
   from_pdf=True
)

In [33]:
pdf_file_paths = ['graph-dag.pdf']

for path in pdf_file_paths:
    print(f"Processing : {path}")
    pdf_result = await kg_builder_pdf.run_async(file_path=os.path.join(ROOT_DIR, "finai", "graph-rag", "data", path))
    print(f"Result: {pdf_result}")

Processing : graph-dag.pdf
Result: run_id='7886da62-24cb-466d-b34a-b7eaebd3b700' result={'resolver': {'number_of_nodes_to_resolve': 124, 'number_of_created_nodes': 110}}


In [28]:
from neo4j_graphrag.indexes import create_vector_index

create_vector_index(neo4j_driver, name="text_embeddings", label="Chunk",
                   embedding_property="embedding", dimensions=1536, similarity_fn="cosine")

In [29]:
from neo4j_graphrag.retrievers import VectorRetriever

vector_retriever = VectorRetriever(
   neo4j_driver,
   index_name="text_embeddings",
   embedder=embedder,
   return_properties=["text"],
)

In [30]:
from neo4j_graphrag.llm import OpenAILLM as LLM
from neo4j_graphrag.generation import RagTemplate
from neo4j_graphrag.generation.graphrag import GraphRAG

llm = get_llm(type=Models.MISTRAL_AI)

rag_template = RagTemplate(template='''Answer the Question using the following Context. Only respond with information mentioned in the Context. Do not inject any speculative information not mentioned.

# Question:
{query_text}

# Context:
{context}

# Answer:
''', expected_inputs=['query_text', 'context'])

v_rag  = GraphRAG(llm=llm, retriever=vector_retriever, prompt_template=rag_template)
# vc_rag = GraphRAG(llm=llm, retriever=vc_retriever, prompt_template=rag_template)

In [31]:
q = "What are the consequences to the french economy considering an anti-euro party wins the election. And tell me the two political parties that are battling each other for the presidency and also the main persons."

print(f"Vector Response: \n{v_rag.search(q, retriever_config={'top_k':5}).answer}")
print("\n===========================\n")

Vector Response: 
The consequences to the French economy if an anti-euro party wins the election could include a short-term economic downturn, with potential labor shortages and trade disruptions leading to market stagnation or recession. Additionally, there could be negative market reactions, especially if there are doubts about France's commitment to the European single market. French economic isolation could undermine confidence in the euro, weakening the currency and pushing global investors away from French equities.

The two political parties battling for the presidency are:
- Le Pen (representing a populist, nationalist approach that questions existing EU agreements)
- Macron (representing an economically liberal, pro-European framework)

The main persons are:
- Marine Le Pen
- Emmanuel Macron




In [34]:
q = "What are the consequences to the french economy considering an anti-euro party wins the election. And tell me the two political parties that are battling each other for the presidency and also the main persons."

print(f"Vector Response: \n{v_rag.search(q, retriever_config={'top_k':5})}")
print("\n===========================\n")

Vector Response: 
answer="If an anti-euro party wins the election in France, the immediate consequences to the French economy could include a short-term economic downturn, with potential labor shortages and trade disruptions leading to stagnation or even recession. This could also result in negative market reactions, especially if there are doubts about France’s commitment to the European single market.\n\nThe two political parties battling for the presidency are:\n\n1. Le Pen's party (representing a populist, nationalist approach that questions existing EU agreements)\n2. Macron's party (representing an economically liberal, pro-European framework)\n\nThe main persons are:\n\n1. Le Pen (associated with the anti-euro party)\n2. Macron (associated with the pro-European party)\n\nThe contrast between their agendas sets up a divide between an economically liberal, pro-European stance and a populist, nationalist approach." retriever_result=None




In [36]:
dict(v_rag.search(q, retriever_config={'top_k':5}))

{'answer': "The two political parties battling each other for the presidency are not explicitly mentioned in the provided context. However, the context discusses two contrasting agendas: an economically liberal, pro-European framework and a populist, nationalist approach. The main persons are not specified in the context.\n\nRegarding the consequences to the French economy if an anti-euro party wins the election, the context mentions several potential impacts:\n- A short-term economic downturn due to labor shortages and trade disruptions.\n- Negative market reactions, especially if there are doubts about France's commitment to the European single market.\n- Potential weakening of the euro and a push away from French equities by global investors.\n- Introduces the threat of volatility and market uncertainty.\n- Undermines confidence in France's role within the EU and the stability of the Eurozone.\n- Sends a reassuring signal to investors who value regulatory consistency and see France'

In [ ]:
"What are the consequences to the french economy considering an anti-euro party wins the election. And tell me the two political parties that are battling each other for the presidency and also the main persons."

print(f"Vector Response: \n{v_rag.search(q, retriever_config={'top_k':5})}")
print("\n===========================\n")